In [102]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# from surprise import Reader, Dataset, SVD
# from surprise.model_selection import cross_validate

In [103]:
df1 = pd.read_csv("Sport_car_price.csv")
df1.head()

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
0,Porsche,911,2022,3,379,331,4,"101,200"
1,Lamborghini,Huracan,2021,5.2,630,443,2.8,"274,390"
2,Ferrari,488 GTB,2022,3.9,661,561,3,"333,750"
3,Audi,R8,2022,5.2,562,406,3.2,"142,700"
4,McLaren,720S,2021,4,710,568,2.7,"298,000"


In [104]:
df2 = pd.read_csv("list_cars2.csv")
df2.head()

,Car Make,Car Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0


In [105]:
# Gabungkan Dataset
merged_df = pd.merge(df1, df2, on='Car Make', how='inner')
print("Merged DataFrame:")
print(merged_df)

Merged DataFrame:
      Car Make Car Model_x  Year_x Engine Size (L) Horsepower Torque (lb-ft)  \
0      Porsche         911    2022               3        379            331   
1      Porsche         911    2022               3        379            331   
2      Porsche         911    2022               3        379            331   
3      Porsche         911    2022               3        379            331   
4      Porsche         911    2022               3        379            331   
...        ...         ...     ...             ...        ...            ...   
32030      Kia     Stinger    2022             3.3        368            376   
32031      Kia     Stinger    2022             3.3        368            376   
32032      Kia     Stinger    2022             3.3        368            376   
32033      Kia     Stinger    2022             3.3        368            376   
32034      Kia     Stinger    2022             3.3        368            376   

      0-60 MPH Time (

In [106]:
# Info tentang dataset, termasuk tipe data dan non-null counts
print("Informasi dataset:")
print(merged_df.info())

# Statistik deskriptif untuk setiap kolom numerik
print("\nStatistik deskriptif untuk kolom numerik:")
print(merged_df.describe())

# Cetak daftar kolom dalam dataset
print("\nDaftar kolom dalam dataset:")
print(merged_df.columns)

Informasi dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32035 entries, 0 to 32034
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Car Make                 32035 non-null  object 
 1   Car Model_x              32035 non-null  object 
 2   Year_x                   32035 non-null  int64  
 3   Engine Size (L)          32005 non-null  object 
 4   Horsepower               32035 non-null  object 
 5   Torque (lb-ft)           32034 non-null  object 
 6   0-60 MPH Time (seconds)  32035 non-null  object 
 7   Price (in USD)           32035 non-null  object 
 8   Car Model_y              32035 non-null  object 
 9   Price                    32035 non-null  int64  
 10  Year_y                   32035 non-null  int64  
 11  Kilometer                32035 non-null  int64  
 12  Fuel Type                32035 non-null  object 
 13  Transmission             32035 non-null  object 
 14  Loc

In [107]:
# Data Quality Assessment:
# i) Finding and Processing Missing/Inconsistent/Duplicate Data
# ii) Mixed Data Values/Mismatched Data Types
# iii) Data Outliers

# Menangani Nilai yang Hilang
merged_df.dropna(inplace=True)

# Mengodekan Variabel Kategorikal (One-hot encoding)
df_encoded = pd.get_dummies(merged_df, columns=['Car Make'])

# Mengonversi kolom 'Price' ke float
merged_df['Price (in USD)'] = merged_df['Price (in USD)'].str.replace(',', '').astype(float)

# Mengonversi kolom 'Price (in USD)' ke float
merged_df['Price'] = merged_df['Price'].astype(float)

# Menyekal Fitur-fitur
scaler = StandardScaler()
merged_df[['Price', 'Price (in USD)']] = scaler.fit_transform(merged_df[['Price', 'Price (in USD)']])

# Pemilihan Fitur (Misalnya, menggunakan semua fitur)
X = merged_df.drop('Car Make', axis=1)
y = merged_df['Car Make']

# Pembagian Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [108]:
# i) Mencari dan Memproses Data yang Hilang
missing_data = merged_df.isnull().sum()
print("Data yang Hilang:\n", missing_data)

# ii) Nilai Data Campuran/Tipe Data yang Tidak Cocok
mixed_data = merged_df.applymap(type).nunique()
print("\nTipe Data Campuran:\n", mixed_data)

# iii) Pencarian Pencilan pada Data
# Menghitung z-score untuk kolom numerik
numeric_cols = merged_df.select_dtypes(include=['int', 'float']).columns
z_scores = (merged_df[numeric_cols] - merged_df[numeric_cols].mean()) / merged_df[numeric_cols].std()

# Menandai baris yang memiliki nilai z-score > 3 atau < -3 sebagai outlier
outliers = merged_df[(z_scores > 3) | (z_scores < -3)]
print("\nData Pencilan:\n", outliers)

Data yang Hilang:
 Car Make                   0
Car Model_x                0
Year_x                     0
Engine Size (L)            0
Horsepower                 0
Torque (lb-ft)             0
0-60 MPH Time (seconds)    0
Price (in USD)             0
Car Model_y                0
Price                      0
Year_y                     0
Kilometer                  0
Fuel Type                  0
Transmission               0
Location                   0
Color                      0
Owner                      0
Seller Type                0
Engine                     0
Max Power                  0
Max Torque                 0
Drivetrain                 0
Length                     0
Width                      0
Height                     0
Seating Capacity           0
Fuel Tank Capacity         0
dtype: int64

Tipe Data Campuran:
 Car Make                   1
Car Model_x                1
Year_x                     1
Engine Size (L)            1
Horsepower                 1
Torque (lb-ft)    

In [109]:
# Data Transformation:
# i) Data Aggregation
# ii) Data Normalization
# iii) Feature Selection/Sampling

# i) Agregasi Data
data_diagregasi = merged_df.groupby('Car Make').agg({'Kilometer': 'mean', 'Price': 'sum'}).reset_index()
print("Data yang Diagregasi:\n", data_diagregasi)

# ii) Normalisasi Data
scaler = MinMaxScaler()
data_dinormalisasi = scaler.fit_transform(merged_df[['Kilometer', 'Price']])
df_dinormalisasi = pd.DataFrame(data_dinormalisasi, columns=['Kilometer', 'Price'])
print("\nData yang Dinormalisasi:\n", df_dinormalisasi)

# iii) Seleksi Fitur/Pemilihan Sampel
X = merged_df[['Kilometer', 'Price']]
y = merged_df['Year_x']
selector = SelectKBest(score_func=f_regression, k=1)
X_terpilih = selector.fit_transform(X, y)
print("\nFitur yang Dipilih:\n", X_terpilih)

Data yang Diagregasi:
          Car Make     Kilometer        Price
0            Audi  53718.406780 -2391.399413
1             BMW  41790.022472   497.119861
2       Chevrolet  53120.000000  -256.604153
3         Ferrari   9500.000000   537.047979
4            Ford  56869.489362 -1563.790589
5          Jaguar  45919.769231    14.257097
6             Kia  28266.434783   -15.461819
7           Lexus  31523.600000    32.743919
8        Maserati  40000.000000    39.537489
9   Mercedes-Benz  45079.232704  2151.432586
10         Nissan  67100.133333  -562.353006
11        Porsche  33358.333333  1743.070307
12    Rolls-Royce  47333.333333   146.202828
13         Toyota  69625.552846  -371.803086

Data yang Dinormalisasi:
        Kilometer     Price
0       0.011521  0.459925
1       0.666665  0.095088
2       0.241664  0.148233
3       0.037496  0.261706
4       0.232080  0.148233
...          ...       ...
27916   0.045829  0.034731
27917   0.262497  0.039787
27918   0.112496  0.031887
27919

In [110]:
# Pisahkan variabel independen dan dependen
X = merged_df[['Kilometer', 'Price']]
y = merged_df['Year_x']

# Bagi dataset menjadi training set (80%) dan test set (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bagi training set menjadi training set (75%) dan validation set (25%)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Print jumlah sampel di setiap set
print("Jumlah sampel dalam setiap set:")
print("Training set:", len(X_train))
print("Validation set:", len(X_val))
print("Test set:", len(X_test))

Jumlah sampel dalam setiap set:
Training set: 16752
Validation set: 5584
Test set: 5585


In [111]:
# i) Standardization
scaler_standard = StandardScaler()
X_train_standardized = scaler_standard.fit_transform(X_train)
X_val_standardized = scaler_standard.transform(X_val)
X_test_standardized = scaler_standard.transform(X_test)

# ii) Normalization
scaler_minmax = MinMaxScaler()
X_train_normalized = scaler_minmax.fit_transform(X_train)
X_val_normalized = scaler_minmax.transform(X_val)
X_test_normalized = scaler_minmax.transform(X_test)

# Menampilkan hasil penskalaan
print("Hasil penskalaan:")
print("Standardization - Training set:\n", X_train_standardized)
print("\nStandardization - Validation set:\n", X_val_standardized)
print("\nStandardization - Test set:\n", X_test_standardized)
print("\nNormalization - Training set:\n", X_train_normalized)
print("\nNormalization - Validation set:\n", X_val_normalized)
print("\nNormalization - Test set:\n", X_test_normalized)

Hasil penskalaan:
Standardization - Training set:
 [[-1.36893528  0.52362162]
 [-1.2320227   1.67897419]
 [-1.41872167  2.8655525 ]
 ...
 [-1.51204004  0.60199824]
 [-0.34520259 -0.91307491]
 [ 0.0748701  -0.10089328]]

Standardization - Validation set:
 [[ 1.3817629  -0.53805371]
 [-0.45411032 -0.22891884]
 [-0.64080929 -0.32727993]
 ...
 [ 1.64149228 -1.06108494]
 [ 0.7283165  -1.08762682]
 [-0.88974125  2.20981185]]

Standardization - Test set:
 [[ 0.0997633  -0.30386062]
 [-0.64080929  0.67975034]
 [-0.01847939  0.13329981]
 ...
 [-1.42600293  3.89600208]
 [-0.98309074  2.92800399]
 [ 0.79054949 -1.07045266]]

Normalization - Training set:
 [[0.01916258 0.14966964]
 [0.03749599 0.25596093]
 [0.01249589 0.36512496]
 ...
 [0.         0.15688021]
 [0.15624648 0.01749497]
 [0.21249672 0.09221488]]

Normalization - Validation set:
 [[0.38749745 0.05199655]
 [0.14166309 0.08043666]
 [0.11666299 0.07138753]
 ...
 [0.42227676 0.0038782 ]
 [0.29999708 0.00143637]
 [0.08332951 0.30479747]]



In [112]:
# Content-Based Recommendation-System

genres_str = merged_df['Car Make'].str.split(',').astype(str)

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 4), min_df=1)
tfidf_matrix = tfidf.fit_transform(genres_str)

tfidf_matrix.shape

(27921, 18)

In [113]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [114]:
indices = pd.Series(merged_df.index, index=merged_df['Car Make'])

def genre_recommendations(title, similarity=False):
    
    if similarity == False:
        
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        
        car_indices = [i[0] for i in sim_scores]
        
        return pd.DataFrame({'Car Make': merged_df['Car Model_x'].iloc[car_indices].values,
                             'Price': merged_df['Price (in USD)'].iloc[car_indices].values})
    
    elif similarity == True:
        
        idx = indices[title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        
        anime_indices = [i[0] for i in sim_scores]
        similarity_ = [i[1] for i in sim_scores]
        
        return pd.DataFrame({'Car Make': merged_df['Car Model_x'].iloc[anime_indices].values,
                             'similarity': similarity_,
                             'Price': merged_df['Price (in USD)'].iloc[anime_indices].values})

In [115]:
# Make a recommendation based on cosine-similarity

indices = pd.Series(merged_df.index, index=merged_df['Car Make'])

def car_recommendations(title, highest_rating=False, similarity=False):
    
    if highest_rating == False:
        if similarity == False:
        
            idx = indices[title]
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
        
            anime_indices = [i[0] for i in sim_scores]
        
            return pd.DataFrame({'Car Make': merged_df['Car Model_x'].iloc[anime_indices].values,
                                 'Price': merged_df['Price'].iloc[anime_indices].values})
    
        elif similarity == True:
        
            idx = indices[title]
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
        
            anime_indices = [i[0] for i in sim_scores]
            similarity_ = [i[1] for i in sim_scores]
        
            return pd.DataFrame({'Car Make': merged_df['Car Model_x'].iloc[Car_indices].values,
                                 'Similarity': similarity_,
                                 'Price': merged_df['Price'].iloc[Car_indices].values})
        
    elif highest_rating == True:
        if similarity == False:
        
            idx = indices[title]
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
        
            Car_indices = [i[0] for i in sim_scores]
        
            result_df = pd.DataFrame({'Car Make': merged_df['Car Model_x'].iloc[Car_indices].values,
                                 'Price': merged_df['Price'].iloc[Car_indices].values,
                                 'Kilometer': merged_df['Kilometer'].iloc[Car_indices].values})
            
            return result_df.sort_values('Rating', ascending=False)
    
        elif similarity == True:
        
            idx = indices[title]
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
        
            Car_indices = [i[0] for i in sim_scores]
            similarity_ = [i[1] for i in sim_scores]
        
            result_df = pd.DataFrame({'Car Make': merged_df['Car Model_x'].iloc[Car_indices].values,
                                 'Similarity': similarity_,
                                 'Price': merged_df['Price'].iloc[Car_indices].values,
                                 'Kilometer': merged_df['Kilometer'].iloc[Car_indices].values})
            
            return result_df.sort_values('Kilometer', ascending=False)

In [116]:
car_recommendations('Porsche', highest_rating=True, similarity=True)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()